**<h1>Import all Neccesary Libraires and Modules</h1>**

In [1]:
import os
import glob

from dotenv import load_dotenv
from openai import OpenAI

import chromadb
from chromadb.utils.embedding_functions import SentenceTransformerEmbeddingFunction
from chromadb.utils.embedding_functions import OpenAIEmbeddingFunction
import chromadb
from chromadb.utils import embedding_functions



import os
from collections import Counter
import tiktoken


from langchain.text_splitter import RecursiveCharacterTextSplitter


from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document
from langchain.chains.summarize import load_summarize_chain
from langchain_openai import ChatOpenAI
import re


**<h1>Query Expansion</h1>**

In [9]:
def augment_query_generated(user_query, model="gpt-3.5-turbo"):
    system_prompt = """You are a helpful expert research assistant. 
    Provide a plausible example answer to the user's query as if you found it in a case document."""
    
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_query},
    ]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
    )
    content = response.choices[0].message.content
    return content



In [10]:
original_query = "I believe I was wrongfully terminated from my job. What can I do?"
hypothetical_answer = augment_query_generated(original_query)
joint_query = f"{original_query} {hypothetical_answer}"

**<h1>Query Vector DB to return most relevant Chunks</h1>**

In [11]:
def query_chroma_collection(chroma_collection, query_text, n_results=1):
    """
    Queries a Chroma collection and returns metadata with relevance scores.

    Args:
        chroma_collection (chromadb.Collection): The Chroma collection to query.
        query_text (str): The query text (original query + hypothetical answer) to search for.
        n_results (int): Number of results to return. Default is 1.

    Returns:
        list of dict: Retrieved metadata with relevance scores included.
    """
    # Query the Chroma collection
    results = chroma_collection.query(
        query_texts=[query_text],
        n_results=n_results,
        include=["documents", "embeddings", "metadatas", "distances"]  # Include distances explicitly
    )

    # Extract metadata and distances from the results
    retrieved_metadata = results["metadatas"][0]
    retrieved_distances = results["distances"][0]

    # Compute relevance scores from distances
    relevance_scores = [1 / (1 + distance) for distance in retrieved_distances]

    # Add relevance scores to the metadata
    for metadata, relevance_score in zip(retrieved_metadata, relevance_scores):
        metadata["relevance_score"] = relevance_score

    return retrieved_metadata

In [12]:
# implementation
retrieved_metadata = query_chroma_collection(chroma_collection, joint_query, n_results=1)
case_doc = retrieved_metadata[0]['source']
print(case_doc)


C14.txt


**<h1>Extract txt file from directory</h1>**

In [14]:
def get_file_contents(pattern, target_filename):
    """
    pattern: A glob pattern, e.g. '/path/to/*.txt'
    target_filename: A filename we want to match in those results
    """
    # Expand the pattern into all matching file paths
    matching_files = glob.glob(pattern)  # returns a list of matching paths

    for fpath in matching_files:
        # Get the base name of the file (e.g., "C2.txt") and compare 
        if os.path.basename(fpath) == target_filename:
            with open(fpath, 'r', encoding='utf-8') as file:
                return file.read()

    # If we exit the loop, the file wasn't found
    raise FileNotFoundError(f"File '{target_filename}' not found in the pattern '{pattern}'.")

In [ ]:
# implementation
file_content = get_file_contents(folder_path, case_doc)
print(file_content)

**<h1>Carryout Text Summarization</h1>**

In [16]:
def summarize_text_with_map_reduce(file_content: str, max_length: int = 100) -> str:
    """
    Summarize long text using the Map-Reduce method, preserving specific details before a list starts.

    Args:
        file_content (str): The long text to summarize.
        max_length (int): The maximum length of the summary in tokens or words (approximation).

    Returns:
        str: The summarized text, preserving details before the first list starts.
    """
    # Step 1: Extract the section to preserve (everything before the first list starts)
    match = re.search(r"^(.*?)(\n\s*1\.)", file_content, re.DOTALL)
    if match:
        preserved_section = match.group(1).strip()
        remaining_content = file_content[len(preserved_section):].strip()
    else:
        # If no list is found, treat the whole text as the preserved section
        preserved_section = file_content.strip()
        remaining_content = ""

    # Step 2: Split the remaining text into chunks
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,  # Adjust chunk size as needed
        chunk_overlap=50  # Adjust overlap to preserve context
    )
    chunks = text_splitter.split_text(remaining_content)

    # Step 3: Convert chunks into a list of Document objects
    documents = [Document(page_content=chunk) for chunk in chunks]

    # Step 4: Initialize the LLM
    llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo")

    # Step 5: Load the Map-Reduce summarization chain
    chain = load_summarize_chain(llm, chain_type="map_reduce")

    # Step 6: Invoke the chain with the documents and adjust the summary length
    result = chain.invoke({"input_documents": documents, "length": max_length})

    # Step 7: Combine the preserved section with the generated summary
    summary = preserved_section + "\n\n" + result["output_text"]

    return summary

In [17]:
summary = summarize_text_with_map_reduce(file_content, max_length=2000)
print(summary)

Central Inland Water Transport Corporation Limited and Another v Brojo Nath Ganguly and Another
Supreme Court of India

6 April 1986
C.A. No. 4412 and 4413 of 1985
The Judgment was delivered by : D. P. Madon, J.

The Appeals by Special Leave granted by the Court address whether a Government company falls under the definition of "the State" as per Article 12 of the Constitution. The Central Inland Water Transport Corporation Limited, a government company, is owned by the Central Government and two State Governments. The Appeals challenge the termination of two employees' services by the Corporation and the validity of Rule 9(i) of the Service Discipline and Appeal Rules. The Calcutta High Court ruled that the Corporation is considered a State under Article 12 and that Rule 9(i) violates Article 14 of the Constitution. The Appeals question whether government companies can be considered part of "the State" and the validity of employment contracts under Article 14. The Court will review th

**<h1>Generate Response</h1>**

In [18]:
def generate_response(question, context_data):
    """
    Generate a response based on the provided legal context (as a string).
    
    Args:
        question (str): The user's legal question.
        context_data (str): The full legal text to be used as context.

    Returns:
        str: A generated response summarizing relevant legal cases.
    """

    # Construct the prompt using the full context_data string
    prompt = f"""You are a legal assistant designed to help users understand their legal situations by retrieving and summarizing relevant cases. Follow these steps STRICTLY:
    
    1. **Sympathize with the user** (1-2 sentences):
       - Acknowledge their situation with empathy (e.g., "I’m sorry to hear...", "This sounds difficult...").
    
    2. **Retrieve and summarize a case** from the knowledge base below:
    {context_data}
       - Format:
         **Case Name**: [Exact case title]
         **Introduction**: [1-2 sentence overview: who was involved and the core issue]
         **Details**: [Key facts/events in chronological order]
         **Verdict**: [Court decision + outcomes like damages or policy changes]

    3. **Next Steps** (3-4 bullet points):
       - Practical actions tied to the case (e.g., "Save emails from [date range]")
       - Resources (e.g., "Contact [Agency Name] within [timeframe]")
    
    Tone Rules:
    - Professional but compassionate
    - Zero legal jargon (avoid terms like "plaintiff" or "motion")
    - If no matching case: 
      * Apologize briefly
      * Provide 2-3 general steps
      * Add: "Every case is unique – consulting a lawyer is recommended"

    Example structure to mimic:
    "I’m sorry to hear about your situation. Let me share a similar case:
    **Case Name**: Smith v. ABC Corp
    **Introduction**: A warehouse worker fired after reporting safety issues.
    **Details**: The employee reported violations in March 2022, was terminated April 2022 with no warning. The employer claimed budget cuts.
    **Verdict**: Court ruled wrongful termination – $150k awarded due to retaliation evidence.
    Next steps:
    - Document all safety reports you filed
    - Contact OSHA within 30 days
    - Consult an employment lawyer"""

    response = client.chat.completions.create(
        model="gpt-4-turbo",
        messages=[
            {"role": "system", "content": prompt},
            {"role": "user", "content": question},
        ],
        temperature=0.3,
        max_tokens=1500
    )

    return response.choices[0].message.content

In [19]:
final_response = generate_response(original_query, summary)
print(final_response)

I’m sorry to hear about your situation. It must be really tough dealing with the uncertainty of wrongful termination. Let me share a similar case that might help you understand how such matters can be approached:

**Case Name**: Central Inland Water Transport Corporation Limited and Another v Brojo Nath Ganguly and Another
**Introduction**: This case involved the Central Inland Water Transport Corporation Limited, a government company, and addressed the wrongful termination of two employees.
**Details**: The employees were terminated under Rule 9(i) of the Service Discipline and Appeal Rules. The Calcutta High Court found that the corporation, being owned by the government, qualified as "the State" under Article 12 of the Constitution and that Rule 9(i) was in violation of Article 14, which ensures equality before the law.
**Verdict**: The court ruled that the termination was invalid and that the rule under which the termination was made violated constitutional provisions.

Next steps: